# WARP

In [9]:
import random
from time import time
from pathlib import Path
from copy import deepcopy
import numpy as np

import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from datasets import load_dataset

from src.reinforce_trainer import ReinforceTrainer
from src.reinforce_config import ReinforceConfig
from src.reinforce_trainer import generate, score_sequence

random.seed(1)
np.random.seed(1)
torch.random.manual_seed(1)

In [2]:
model_name = "lvwerra/gpt2-imdb"
policy = AutoModelForCausalLM.from_pretrained(model_name)
ref_policy = AutoModelForCausalLM.from_pretrained(model_name)
policy_tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="left")
policy_tokenizer.pad_token = policy_tokenizer.eos_token

reward_model_name = "models/reward/checkpoint-391"
# reward_model_name = "lvwerra/distilbert-imdb"
reward_model = AutoModelForSequenceClassification.from_pretrained(reward_model_name)
reward_tokenizer = AutoTokenizer.from_pretrained(reward_model_name)

C:\Users\Юрий\AppData\Roaming\Python\Python39\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [3]:
def tokenize(examples):
    prompt_len = random.randint(5, 15)
    examples["input_ids"] = policy_tokenizer.encode(examples["text"])[:prompt_len]
    examples["query"] = policy_tokenizer.decode(examples["input_ids"])
    return examples

dataset = load_dataset("stanfordnlp/imdb")
prompt_dataset = dataset.map(tokenize, batched=False)

test_dataset = prompt_dataset["test"].select(range(100))
test_dataloader = DataLoader(
    test_dataset.remove_columns(["text", "label", "query"]),
    batch_size=64,
    shuffle=False,
    collate_fn=DataCollatorWithPadding(policy_tokenizer),
)

train_dataset = prompt_dataset["train"].remove_columns(["text", "label", "query"])
eval_dataset = prompt_dataset["test"].remove_columns(["text", "label", "query"])
eval_dataset = eval_dataset.train_test_split(0.1, seed=1)["test"]

In [4]:
training_args = ReinforceConfig(
    output_dir="_",

    per_device_train_batch_size=64,
    local_rollout_forward_batch_size=64,
    gradient_accumulation_steps=1,
    learning_rate=5e-5,
    weight_decay=0.0,
    seed=1,

    rloo_k=1,
    num_ppo_epochs=1,
    num_mini_batches=1,
    ema_beta=0.99,
    kl_coef=1.0,
    temperature=0.7,
)
training_args.total_episodes = 400 * 64

In [5]:
from src.get_reward import BertRewardGetter

trainer = ReinforceTrainer(
    training_args,
    tokenizer=policy_tokenizer,
    policy=policy,
    ref_policy=ref_policy,
    reward_model=reward_model,
    get_reward_func=BertRewardGetter(policy_tokenizer, reward_tokenizer).get_reward,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)

In [ ]:
timestamp = int(time())
trainer.args.output_dir = f"models/reinforce/{timestamp}"
save_path = Path(trainer.args.output_dir)
trainer.train()

policy.save_pretrained(save_path / "policy", from_pt=True)
ref_policy.save_pretrained(save_path / "ref_policy", from_pt=True)
trainer.save_state()
with open(save_path / "args.json") as f:
    f.write(trainer.args.to_json_string())

## Method

In [ ]:
from src.model_ops import model_add_, model_sub_, model_lerp_, model_slerp_

training_args.per_device_train_batch_size = 64
training_args.total_episodes = 100 * training_args.per_device_train_batch_size
training_args.ema_beta = 0.99
training_args.iterations = 2
training_args.runs = 2
training_args.eta = 0.5

policy_init = AutoModelForCausalLM.from_pretrained(model_name).cuda()
timestamp = int(time())

for i in range(training_args.iterations):
    for m in range(training_args.runs):
        # initialize policies
        policy_m = deepcopy(policy_init)
        policy_ref_m = deepcopy(policy_init)

        # perform training
        train_dataset = train_dataset.shuffle(seed=1)
        trainer = ReinforceTrainer(
            training_args,
            tokenizer=policy_tokenizer,
            policy=policy_m,
            ref_policy=policy_ref_m,
            reward_model=reward_model,
            get_reward_func=BertRewardGetter(policy_tokenizer, reward_tokenizer).get_reward,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset
        )
        trainer.args.output_dir = f"models/warp/{timestamp}/i{i+1}m{m+1}"
        trainer.train()
        policy_m.save_pretrained(Path(trainer.args.output_dir) / "policy", from_pt=True)
        policy_ref_m.save_pretrained(Path(trainer.args.output_dir) / "ref_policy", from_pt=True)
        trainer.save_state()

        # compute task vector inplace
        model_sub_(policy_m, policy_init)
        # collect slerp iterations inplace
        if m == 0:
            policy_slerp = policy_m
        else:
            model_slerp_(policy_slerp, policy_m, coeff=1/training_args.runs)
    # add init back to complete slerp
    model_add_(policy_slerp, policy_init)
    # update init inplace
    model_lerp_(policy_init, policy_slerp, coeff=training_args.eta)

with open(f"models/warp/{timestamp}/args.json") as f:
    f.write(trainer.args.to_json_string())
policy_init.save_pretrained(f"models/warp/{timestamp}/final/policy", from_pt=True)

## Evaluation

In [13]:
def evaluate_policy(policy, ref_policy, test_dataloader):
    rewards = 0
    kls = 0
    total = 0
    with torch.no_grad():
        for data in test_dataloader:
            prompts = data["input_ids"].to(policy.device)
            prompt_length = prompts.shape[1]
            prompts_responses, _ = generate(
                policy,
                prompts,
                policy_tokenizer.pad_token_id,
                trainer.generation_config,
            )
            logprobs = score_sequence(
                policy,
                prompts_responses,
                prompt_length,
                pad_token_id=policy_tokenizer.pad_token_id,
                temperature=training_args.temperature
            )
            ref_logprobs = score_sequence(
                ref_policy,
                prompts_responses,
                prompt_length,
                pad_token_id=policy_tokenizer.pad_token_id,
                temperature=training_args.temperature
            )
            _, reward, _ = trainer.get_reward(
                reward_model, prompts_responses, policy_tokenizer.pad_token_id, prompt_length
            )
            kl = (logprobs.exp() * (logprobs - ref_logprobs)).sum(1)

            rewards += reward.sum().item()
            kls += kl.sum().item()
            total += prompts.shape[0]
    rewards /= total
    kls /= total
    return rewards, kls

In [ ]:
policy = AutoModelForCausalLM.from_pretrained("models/warp/1722875607/final/policy").cuda()
ref_poicy = AutoModelForCausalLM.from_pretrained(model_name).cuda()

rs, kls = 0, 0
for _ in range(10):
    r, kl = evaluate_policy(policy, ref_policy, test_dataloader)
    rs += r
    kls += kl
rs /= 10
kls /= 10
print(rs, kls)